In [3]:
import pandas as pd
import numpy as np

from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore")

In [4]:
raw_data = pd.read_excel('../../data/market_data.xlsx')

In [3]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63457 entries, 0 to 63456
Data columns (total 13 columns):
 #   Column                                                  Non-Null Count  Dtype         
---  ------                                                  --------------  -----         
 0   Time (UTC+10)                                           63457 non-null  datetime64[ns]
 1   Regions NSW Trading Price ($/MWh)                       63457 non-null  float64       
 2   Regions SA Trading Price ($/MWh)                        63457 non-null  float64       
 3   Regions TAS Trading Price ($/MWh)                       63457 non-null  float64       
 4   Regions VIC Trading Price ($/MWh)                       63457 non-null  float64       
 5   Regions NSW Trading Total Intermittent Generation (MW)  63457 non-null  float64       
 6   Regions SA Trading Total Intermittent Generation (MW)   63457 non-null  float64       
 7   Regions TAS Trading Total Intermittent Generation (MW)  63

In [5]:
raw_data['time'] = raw_data['Time (UTC+10)'].dt.time
timelist = [i for i in raw_data['time'].unique()]
raw_data.loc[raw_data['time'] == timelist[0], ['period']] = 48

for i in range(1,48):
    raw_data.loc[raw_data['time'] == timelist[i], ['period']] = i

In [6]:
raw_data['day'] = raw_data['Time (UTC+10)'].dt.weekday+1

In [7]:
raw_data['month'] = raw_data['Time (UTC+10)'].dt.month

In [9]:
raw_data['month'] = raw_data['Time (UTC+10)'].dt.month

In [10]:
SUMMER = [12,1,2]
AUTUMN = [3,4,5]
WINTER = [6,7,8]
SPRING = [9,10,11]



for i in list(raw_data.index):
    if ((raw_data.at[i, 'month'] in SUMMER)):
        raw_data.at[i,'season'] = 0
    elif ((raw_data.at[i,'month'] in AUTUMN)):
        raw_data.at[i,'season'] = 1
    elif ((raw_data.at[i,'month'] in WINTER)):
        raw_data.at[i,'season'] = 2
    else:
        raw_data.at[i,'season'] = 3

In [18]:
X_pricepred_vic = raw_data[['Regions VIC Operational Demand (MW)', 'Regions VIC Trading Total Intermittent Generation (MW)']]
#X_pricepred_vic = raw_data[['Regions VIC Operational Demand (MW)', 'Regions VIC Trading Total Intermittent Generation (MW)', 'period']]
#X_pricepred_vic = raw_data[['Regions VIC Operational Demand (MW)', 'Regions VIC Trading Total Intermittent Generation (MW)', 'period', 'day']]
#X_pricepred_vic = raw_data[['Regions VIC Operational Demand (MW)', 'Regions VIC Trading Total Intermittent Generation (MW)', 'period', 'day', 'season']]



y_price_pred_vic = raw_data[['Regions VIC Trading Price ($/MWh)']]

In [16]:
X_pricepred_vic

,Regions VIC Operational Demand (MW),period
0,4599,48.0
1,4398,1.0
2,4238,2.0
3,4112,3.0
4,3956,4.0
...,...,...
63452,5492,44.0
63453,5344,45.0
63454,5204,46.0
63455,5268,47.0


In [17]:
y_price_pred_vic

,Regions VIC Trading Price ($/MWh)
0,90.43
1,92.46
2,87.62
3,73.08
4,70.18
...,...
63452,49.93
63453,62.86
63454,32.26
63455,25.10


In [12]:
# with "day" parameter, score: 0.05101117241840025
# without "day" parameter, score: 0.049700438463686325
# need demand, without demand param, score = 0.004
# with "season" parameter, score: 0.051325919419076604

from sklearn.linear_model import LinearRegression
reg = LinearRegression()
reg.fit(X_pricepred_vic, y_price_pred_vic)
reg.score(X_pricepred_vic, y_price_pred_vic)

0.051325919419076604

https://www.analyticsvidhya.com/blog/2021/08/vector-autoregressive-model-in-python/

In [1]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import adfuller
from statsmodels.tools.eval_measures import rmse, aic

In [19]:
model = VAR(X_pricepred_vic)

In [20]:
for i in range(1,50):
    model = VAR(X_pricepred_vic)
    results = model.fit(i)
    print('Order =', i)
    print('AIC: ', results.aic)
    print('BIC: ', results.bic)
    print()

Order = 1
AIC:  15.711222448166406
BIC:  15.71207892528737

Order = 2
AIC:  14.498309875328138
BIC:  14.499737357208563

Order = 3
AIC:  14.488379907117228
BIC:  14.490378409767665

Order = 4
AIC:  14.486360357754888
BIC:  14.488929897186598

Order = 5
AIC:  14.474018660885383
BIC:  14.477159253110335

Order = 6
AIC:  14.471154048626277
BIC:  14.47486570965715

Order = 7
AIC:  14.47080825617071
BIC:  14.475091002020894

Order = 8
AIC:  14.469893132259276
BIC:  14.474746978942875

Order = 9
AIC:  14.46877955337566
BIC:  14.47420451690748

Order = 10
AIC:  14.467302573348382
BIC:  14.473298669743947

Order = 11
AIC:  14.46610673560616
BIC:  14.472673980881702

Order = 12
AIC:  14.46519298136357
BIC:  14.472331391536027

Order = 13
AIC:  14.460857593666377
BIC:  14.468567184753402

Order = 14
AIC:  14.460784721059609
BIC:  14.469065509079565

Order = 15
AIC:  14.459835375031748
BIC:  14.46868737600371

Order = 16
AIC:  14.434378748714641
BIC:  14.443801978658389

Order = 17
AIC:  14.43427

In [21]:
result = model.fit(49)
result.summary()

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Fri, 01, Oct, 2021
Time:                     18:27:02
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    13.9397
Nobs:                     63408.0    HQIC:                   13.9202
Log likelihood:          -620794.    FPE:                1.10067e+06
AIC:                      13.9114    Det(Omega_mle):     1.09725e+06
--------------------------------------------------------------------
Results for equation Regions VIC Operational Demand (MW)
                                                                coefficient       std. error           t-stat            prob
-----------------------------------------------------------------------------------------------------------------------------
const                                                             62.251787         2.353396          